In [ ]:
#  Tell me about a situation where you solved a very complex problem.

# 1 Amazon Cart discount and scalable

Write code that will be used by a Shopping cart service to enforce rules on the order

eg.

Offer free 2 day shipping on orders > $35 if customer is not a prime member

Offer free 2 day shipping on all orders if customer is a prime member

Offer free 1 day shipping for order that are > $125

Offer free 2 hour shipping for prime customer that have > $25 and the items are grocery items

Make this extensible to add other rules in the future

Apply a 10% discount if an item has been marked for subscribe and save

In [ ]:
from typing import List

class Order:
    def __init__(self, total_amount: float, items: List[str], is_prime: bool, is_grocery: bool, subscribe_and_save: bool):
        self.total_amount = total_amount
        self.items = items
        self.is_prime = is_prime
        self.is_grocery = is_grocery
        self.subscribe_and_save = subscribe_and_save
        self.shipping_type = 'Standard'
        self.discount = 0.0


class Rule:
    def apply(self, order: Order):
        raise NotImplementedError

class FreeTwoDayShippingRule(Rule):
    def apply(self, order: Order):
        if order.is_prime or order.total_amount > 35:
            order.shipping_type = 'Free 2 Day Shipping'

class FreeOneDayShippingRule(Rule):
    def apply(self, order: Order):
        if order.total_amount > 125:
            order.shipping_type = 'Free 1 Day Shipping'

class FreeTwoHourShippingRule(Rule):
    def apply(self, order: Order):
        if order.is_prime and order.is_grocery and order.total_amount > 25:
            order.shipping_type = 'Free 2 Hour Shipping'

class SubscribeAndSaveDiscountRule(Rule):
    def apply(self, order: Order):
        if order.subscribe_and_save:
            order.discount = 0.10 * order.total_amount


class ShoppingCartService:
    def __init__(self):
        self.rules = []

    def add_rule(self, rule: Rule):
        self.rules.append(rule)
        rule.apply(order)


# Example usage:
order = Order(total_amount=150, items=['item1', 'item2'], is_prime=False, is_grocery=False, subscribe_and_save=True)

service = ShoppingCartService()

service.add_rule(FreeTwoDayShippingRule())
service.add_rule(FreeOneDayShippingRule())
service.add_rule(FreeTwoHourShippingRule())
service.add_rule(SubscribeAndSaveDiscountRule())

# service.apply_rules(order)

print(f"Shipping Type: {order.shipping_type}")
print(f"Discount: {order.discount}")
print(f"Total after Discount: {order.total_amount - order.discount}")

In [ ]:
# # 1 Amazon Cart discount and scalable (Selft)

# Write code that will be used by a Shopping cart service to enforce rules on the order

# eg.

# Offer free 2 day shipping on orders > $35 if customer is not a prime member

# Offer free 2 day shipping on all orders if customer is a prime member

# Offer free 1 day shipping for order that are > $125

# Offer free 2 hour shipping for prime customer that have > $25 and the items are grocery items

# Make this extensible to add other rules in the future

# Apply a 10% discount if an item has been marked for subscribe and save


class Order:

    def __init__( self, total, items,  is_groc, is_fav,is_prime=True,) -> None:
        
        self.total, self.items, self.is_prime, self.is_groc, self.is_fav = total, items, is_prime, is_groc, is_fav
        self.discount = 0
        self.deltype = 'Regular'


class Rule:
    def apply(self, order: Order):
        raise NotImplemented

class rule1(Rule):
    def apply(self, order: Order):
        if order.total > 35 or order.is_prime:
            order.deltype = 'two day'
        
class rule2(Rule):
    def apply(self, order):
        if order.total >125:
            order.deltype = 'one day'

class rule3(Rule):
    def apply(self, order):
        if order.total >25 and order.is_groc:
            order.deltype = 'one hour'


class rule4(Rule):
    def apply(self, order):
        if order.is_fav:
            order.discount = 0.1 * order.total

       
order = Order(total=35, items=['item1', 'item2'], is_groc = True, is_fav = True)

rule1().apply(order)
rule2().apply(order)
rule3().apply(order)
rule4().apply(order)

print(f"Order Total: {order.total}")
print(f"Items: {order.items}")
print(f"Is Prime Member: {order.is_prime}")
print(f"Is Grocery: {order.is_groc}")
print(f"Subscribe and Save: {order.is_fav}")
print(f"Shipping Type: {order.deltype}")
print(f"Discount: {order.discount}")
print(f"Total after Discount: {order.total - order.discount}")


In [ ]:
from abc import ABC, abstractmethod
from typing import List

class File:
    def __init__(self, name: str, extension: str, size: int):
        self.name = name
        self.extension = extension
        self.size = size

    def get_name(self) -> str:
        return self.name

    def get_extension(self) -> str:
        return self.extension

    def get_size(self) -> int:
        return self.size

class FileSystem:
    def __init__(self, name: str, is_directory: bool = False, sub_directories: List['FileSystem'] = None, files: List[File] = None):
        self.name = name
        self.is_directory = is_directory
        self.sub_directories = sub_directories if sub_directories else []
        self.files = files if files else []


class Filter(ABC):
    @abstractmethod
    def match(self, file: File) -> bool:
        pass

class NameFilter(Filter):
    def __init__(self, name: str):
        self.name = name

    def match(self, file: File) -> bool:
        return file.get_name() == self.name

class SizeFilter(Filter):
    def __init__(self, size: int, operator: str):
        self.size = size
        self.operator = operator

    def match(self, file: File) -> bool:
        return eval(str(file.get_size()) + str(self.operator) + str(self.size))

class ExtensionFilter(Filter):
    def __init__(self, extension: str):
        self.extension = extension

    def match(self, file: File) -> bool:
        return file.get_extension() == self.extension


class Search:
    def __init__(self, file_system: FileSystem, filters: List[Filter], condition: str = "AND"):
        self.file_system = file_system
        self.filters = filters
        self.condition = condition

    def check_conditions(self, file: File) -> bool:
        if self.condition == "AND":
            for f in self.filters:
                if not f.match(file):
                    return False
            return True
        
        elif self.condition == "OR":
            for f in self.filters:
                if f.match(file):
                    return True
            return False
        
        return False

    def find_files(self) -> List[str]:
        queue = [self.file_system]
        result = []

        # BFS
        while queue:
            node = queue.pop(0)
            for file in node.files:
                if self.check_conditions(file):
                    result.append(file.get_name())
            for sub_directory in node.sub_directories:
                queue.append(sub_directory)

        return result

if __name__ == "__main__":
    # Simulation
    f1 = File("abc", "txt", 10)
    f2 = File("cde", "txt", 20)
    f3 = File("def", "pdf", 30)
    f4 = File("ghi", "py", 5)
    f5 = File("uvw", "java", 10)

    directory_files = [f1, f2, f3, f4, f5]
    file_system = FileSystem("/", True, [], directory_files)

    # Example searches
    res = Search(file_system, [NameFilter("abc")]).find_files()
    print("Files with name 'abc':", res)

    res = Search(file_system, [SizeFilter(10, ">=")]).find_files()
    print("Files with size >= 10:", res)

    res = Search(file_system, [ExtensionFilter("java"), SizeFilter(10, ">=")], "OR").find_files()
    print("Files with extension 'java' OR size >= 10:", res)


In [ ]:
from abc import ABC, abstractmethod

class VehicleSize:
    MOTORCYCLE = 'Motorcycle'
    CAR = 'Car'
    BUS = 'Bus'

class Vehicle(ABC):
    def __init__(self, license_plate, size):
        self.license_plate = license_plate
        self.size = size

    @abstractmethod
    def can_fit_in_spot(self, spot):
        pass

class Motorcycle(Vehicle):
    def __init__(self, license_plate):
        super().__init__(license_plate, VehicleSize.MOTORCYCLE)

    def can_fit_in_spot(self, spot):
        return True  # A motorcycle can park anywhere

class Car(Vehicle):
    def __init__(self, license_plate):
        super().__init__(license_plate, VehicleSize.CAR)

    def can_fit_in_spot(self, spot):
        return spot.size in [VehicleSize.CAR, VehicleSize.BUS]

class Bus(Vehicle):
    def __init__(self, license_plate):
        super().__init__(license_plate, VehicleSize.BUS)

    def can_fit_in_spot(self, spot):
        return spot.size == VehicleSize.BUS

class ParkingSpot:
    def __init__(self, number, size):
        self.number = number
        self.size = size
        self.vehicle = None

    def park(self, vehicle):
        if self.is_available() and vehicle.can_fit_in_spot(self):
            self.vehicle = vehicle
            return True
        return False

    def remove_vehicle(self):
        self.vehicle = None

    def is_available(self):
        return self.vehicle is None

class ParkingLot:
    def __init__(self, num_spots):
        self.spots = [ParkingSpot(i, self.assign_size(i)) for i in range(num_spots)]

    def assign_size(self, index):
        if index % 3 == 0:
            return VehicleSize.BUS
        elif index % 2 == 0:
            return VehicleSize.CAR
        else:
            return VehicleSize.MOTORCYCLE

    def park_vehicle(self, vehicle):
        for spot in self.spots:
            if spot.park(vehicle):
                return True
        return False

    def remove_vehicle(self, vehicle):
        for spot in self.spots:
            if spot.vehicle == vehicle:
                spot.remove_vehicle()
                return True
        return False

# Example usage
parking_lot = ParkingLot(30)
car = Car("CAR123")
motorcycle = Motorcycle("MOTO456")
bus = Bus("BUS789")

print(parking_lot.park_vehicle(car))        # Attempts to park car
print(parking_lot.park_vehicle(motorcycle)) # Attempts to park motorcycle
print(parking_lot.park_vehicle(bus))        # Attempts to park bus

parking_lot.remove_vehicle(car)             # Remove the car from the spot


In [ ]:
class TicTacToe:
    def __init__(self, n):
        self.n = n
        self.board = [[0] * n for _ in range(n)]
        self.rowSum = [0] * n
        self.colSum = [0] * n
        self.diagSum = 0
        self.revDiagSum = 0

    def move(self, player, row, col):
        if not (0 <= row < self.n and 0 <= col < self.n):
            raise ValueError("Move out of board boundary!")
        if self.board[row][col] != 0:
            raise ValueError("Square is already occupied!")
        if player not in (1, 2):
            raise ValueError("Invalid player!")

        # Player 1 is +1, Player 2 is -1 (to easily calculate sums)
        value = 1 if player == 1 else -1
        self.board[row][col] = value

        # Update rowSum, colSum, and diagonals
        self.rowSum[row] += value
        self.colSum[col] += value
        if row == col:
            self.diagSum += value
        if row + col == self.n - 1:
            self.revDiagSum += value

        # Check if this move wins the game
        if (abs(self.rowSum[row]) == self.n or
            abs(self.colSum[col]) == self.n or
            abs(self.diagSum) == self.n or
            abs(self.revDiagSum) == self.n):
            return player

        return 0

# Example usage
game = TicTacToe(3)
print(game.move(2, 0, 0))  # Player 1 moves at (0, 0)
print(game.move(1, 0, 1))  # Player 2 moves at (0, 1)
print(game.move(2, 1, 1))  # Player 1 moves at (1, 1)
print(game.move(1, 0, 2))  # Player 2 moves at (0, 2)
print(game.move(2, 2, 2))  # Player 1 moves at (2, 2), potentially wins


In [30]:
class File:
    name = 'abc'

    def __init__(self,fname) -> None:
        print('called')
        self.fname = fname
        # self.name = 'ram'
        type(self).name = 'UPDATE CLASS VAR'


    def open(self,flag):
        print('opend with flag')

class Jenga(File):
    pass

f1 = Jenga('first file')

called


In [23]:
f1.fname
f1.name
f1.open('yes')

opend with flag


In [27]:
class CustomListIterator:
    def __init__(self, custom_list):
        self._custom_list = custom_list
        self._index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self._index < len(self._custom_list):
            result = self._custom_list[self._index]
            self._index += 1
            return result
        raise StopIteration

class CustomList:
    def __init__(self, elements):
        self._elements = elements

    def __iter__(self):
        return CustomListIterator(self._elements)

# Example usage:
custom_list = CustomList([1, 2, 3, 4])
for item in custom_list:
    print(item)


1
2
3
4


In [28]:
from typing import List

class Product:
    def __init__(self, name: str, category: str, weight: float):
        self.name = name
        self.category = category
        self.weight = weight

class Warehouse:
    def __init__(self, allowed_categories: List[str], max_capacity: float):
        self.allowed_categories = allowed_categories
        self.max_capacity = max_capacity
        self.current_capacity = 0
        self.products = []

    def add_product(self, product: Product) -> bool:
        if self.check_constraints(product):
            self.products.append(product)
            self.current_capacity += product.weight
            return True
        return False

    def remove_product(self, product_name: str) -> bool:
        for product in self.products:
            if product.name == product_name:
                self.products.remove(product)
                self.current_capacity -= product.weight
                return True
        return False

    def check_constraints(self, product: Product) -> bool:
        if product.category not in self.allowed_categories:
            print(f"Product category '{product.category}' is not allowed in this warehouse.")
            return False
        if self.current_capacity + product.weight > self.max_capacity:
            print(f"Adding product '{product.name}' exceeds warehouse capacity.")
            return False
        return True

# Example usage:
allowed_categories = ["electronics", "furniture"]
max_capacity = 1000.0  # Example capacity in weight units

warehouse = Warehouse(allowed_categories, max_capacity)

product1 = Product("Laptop", "electronics", 2.5)
product2 = Product("Table", "furniture", 30.0)
product3 = Product("Phone", "electronics", 1.0)
product4 = Product("Couch", "home_decor", 100.0)  # Invalid category

print(warehouse.add_product(product1))  # Output: True
print(warehouse.add_product(product2))  # Output: True
print(warehouse.add_product(product3))  # Output: True
print(warehouse.add_product(product4))  # Output: False

print(warehouse.remove_product("Table"))  # Output: True
print(warehouse.remove_product("Chair"))  # Output: False


True
True
True
Product category 'home_decor' is not allowed in this warehouse.
False
True
False


In [29]:
class Transaction:
    def __init__(self, amount: float, sender: str, receiver: str, timestamp: str):
        self.amount = amount
        self.sender = sender
        self.receiver = receiver
        self.timestamp = timestamp

from abc import ABC, abstractmethod

class Rule(ABC):
    @abstractmethod
    def is_valid(self, transaction: Transaction) -> bool:
        pass

class AmountRule(Rule):
    def __init__(self, min_amount: float, max_amount: float):
        self.min_amount = min_amount
        self.max_amount = max_amount
    def is_valid(self, transaction: Transaction) -> bool:
        return self.min_amount <= transaction.amount <= self.max_amount

class SenderRule(Rule):
    def __init__(self, allowed_senders: set):
        self.allowed_senders = allowed_senders
    def is_valid(self, transaction: Transaction) -> bool:
        return transaction.sender in self.allowed_senders

class ReceiverRule(Rule):
    def __init__(self, allowed_receivers: set):
        self.allowed_receivers = allowed_receivers
    def is_valid(self, transaction: Transaction) -> bool:
        return transaction.receiver in self.allowed_receivers

class TransactionValidator:
    def __init__(self, rules: list):
        self.rules = rules
    def is_valid(self, transaction: Transaction) -> bool:
        return all(rule.is_valid(transaction) for rule in self.rules)

# Example usage:
allowed_categories = ["electronics", "furniture"]
max_capacity = 1000.0  # Example capacity in weight units

validator = TransactionValidator(rules=[
    AmountRule(min_amount=10, max_amount=1000),
    SenderRule(allowed_senders={'Alice', 'Bob'}),
    ReceiverRule(allowed_receivers={'Charlie', 'David'})
])

transaction1 = Transaction(amount=50, sender='Alice', receiver='Charlie', timestamp='2024-07-21T10:00:00Z')
transaction2 = Transaction(amount=5000, sender='Alice', receiver='Charlie', timestamp='2024-07-21T10:00:00Z')
transaction3 = Transaction(amount=50, sender='Eve', receiver='Charlie', timestamp='2024-07-21T10:00:00Z')

print(validator.is_valid(transaction1))  # Output: True
print(validator.is_valid(transaction2))  # Output: False
print(validator.is_valid(transaction3))  # Output: False


True
False
False


In [30]:
def count_sequence_in_array(sequence, array):
    def find_sequence(seq, arr, start_idx):
        if not seq:
            return 1
        if start_idx >= len(arr):
            return 0
        
        count = 0
        for i in range(start_idx, len(arr)):
            if arr[i] == seq[0]:
                count += find_sequence(seq[1:], arr, i + 1)
        return count

    return find_sequence(sequence, array, 0)

# Example usage:
sequence = [4, 6, 2]
array = [3, 4, 4, 6, 7, 8, 2, 6, 9, 2]

print(count_sequence_in_array(sequence, array))  # Output: 1


6


In [31]:
import sys

def floyd_warshall(num_nodes, edges):
    # Step 1: Initialize the adjacency matrix
    inf = sys.maxsize
    dist = [[inf] * num_nodes for _ in range(num_nodes)]
    
    for i in range(num_nodes):
        dist[i][i] = 0

    # Step 2: Update the matrix with the given edges
    for edge in edges:
        u, v, w = edge
        dist[u][v] = w

    # Step 3: Floyd-Warshall Algorithm
    for k in range(num_nodes):
        for i in range(num_nodes):
            for j in range(num_nodes):
                if dist[i][j] > dist[i][k] + dist[k][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
    
    return dist

# Example usage:
num_nodes = 4
edges = [
    [0, 1, 3],
    [0, 2, 6],
    [1, 0, 3],
    [1, 2, 2],
    [1, 3, 1],
    [2, 0, 6],
    [2, 1, 2],
    [2, 3, 1],
    [3, 1, 1],
    [3, 2, 1]
]

distances = floyd_warshall(num_nodes, edges)

# Print the result
for row in distances:
    print(row)


[0, 3, 5, 4]
[3, 0, 2, 1]
[5, 2, 0, 1]
[4, 1, 1, 0]


In [2]:
import heapq
from collections import defaultdict, deque
from datetime import datetime, timedelta

class TimeWindowFrequency:
    def __init__(self, k, window_size="1 hour"):
        self.k = k
        self.items = defaultdict(int)
        self.time_queue = deque()
        self.window_size = timedelta(hours=1)  # Configure as needed

    def process_item(self, item_id, timestamp):
        # Parse timestamp if in string format, assuming ISO format
        if isinstance(timestamp, str):
            timestamp = datetime.fromisoformat(timestamp)

        # Push the new item into the queue and map
        self.items[item_id] += 1
        self.time_queue.append((item_id, timestamp))
        
        # Remove items from the front of the queue if they're outside the time window
        current_time = timestamp
        while self.time_queue and (current_time - self.time_queue[0][1] > self.window_size):
            old_item, old_time = self.time_queue.popleft()
            self.items[old_item] -= 1
            if self.items[old_item] == 0:
                del self.items[old_item]

    def get_k_most_frequent(self):
        # Extract top k items by frequency using a heap
        # Negate the count to use min-heap as max-heap
        if len(self.items) <= self.k:
            return sorted(((key, self.items[key]) for key in self.items), key=lambda x: -x[1])
        return heapq.nlargest(self.k, self.items.items(), key=lambda x: x[1])

# Example usage
tracker = TimeWindowFrequency(k=3)
data = [
    ('item1', '2024-06-22T17:07:00'), ('item2', '2024-07-22T17:08:00'), ('item1', '2024-07-22T17:09:00'),
    ('item3', '2024-07-22T17:10:00'), ('item2', '2024-07-22T17:11:00'), ('item2', '2024-07-22T17:12:00'),
    ('item3', '2024-07-22T17:59:00'), ('item4', '2024-07-22T18:00:00')
]

for item, timestamp in data:
    tracker.process_item(item, timestamp)

print("K most frequent items in the last hour:", tracker.get_k_most_frequent())


K most frequent items in the last hour: [('item2', 3), ('item3', 2), ('item1', 1)]


In [10]:
def min_steps_to_one(N):
    steps = 0
    while N > 1:
        if N % 2 == 0:
            N //= 2
        else:
            N -= 1
        steps += 1
    return steps

ind = -1
maxi = 0
for i in range(10000000):
    
    k = min_steps_to_one(i)
    # print(f'number = {i} min steps to reach: {k}')
    if k>maxi:
        ind = i
        maxi = k

print(ind,maxi)


8388607 44
